### Chapter 9: 2D axissymmetric tri element

Isoparametric formulation

In [1]:
import numpy as np
import sympy as sp
sp.init_printing()

### The 2D axissymmetric tri element

This is a triangular element with nodes: i, j, m.

Coordinates are: r, z for radial and vertical directions, respectively.

Each node hat 2 degrees of freedom (DOFs):
- u: radial displacement
- w: vertical displacement

The DOFs are ordered as follows: u, w.

### The physical properties of the element

In [2]:

### The physical properties of the element
E = 200e9  # Young's modulus in Pa
nu = 0.3  # Poisson's ratio

# Define symbolic physical coordinates
r1, z1, r2, z2, r3, z3 = sp.symbols('r1 z1 r2 z2 r3 z3')

# Example coordinates for the nodes
# physical_coords = {'P1': (0, 0),
#                    'P2': (1, 0),
#                    'P3': (0, 1)}  # example coordinates for the nodes
physical_coords = {'P1': (3, 0),
                   'P2': (4, 0),
                   'P3': (3, 1)}  # example coordinates for the nodes
# physical_coords = {'P1': (0, 0),
#                    'P2': (3, 0),
#                    'P3': (0, 2)}  # example coordinates for the nodes

midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0]) / 3, \
            (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1]) / 3

physical_coords_map = {'r1': physical_coords['P1'][0], 'z1': physical_coords['P1'][1],
                       'r2': physical_coords['P2'][0], 'z2': physical_coords['P2'][1],
                       'r3': physical_coords['P3'][0], 'z3': physical_coords['P3'][1]}

# Define symbolic variables. Capital letters R, Z are the symbols for the variables r, z
R, Z = sp.symbols('R Z')  # radial and vertical coordinates

# Define the physical nodes as a matrix so it can be used a symbolic evaluation
physical_nodes = sp.Matrix([[r1, z1],
                            [r2, z2],
                            [r3, z3]])

In [3]:
ND = 2  # number of degrees of freedom per node
NNODE = 3  # number of nodes

### The isoparametric part

In [4]:
# coordinates of the nodes in the element
n1 = np.array([0, 0])  # node 1 coordinates
n2 = np.array([1, 0])   # node 2 coordinates
n3 = np.array([0, 1])    # node 3 coordinates
natural_nodes = np.array([n1, n2, n3])  # all nodes

### Shape functions and their derivatives

In [5]:
# Define the shape functions symbolically
def shape_functions() -> sp.Matrix:
    """shape functions"""

    return sp.Matrix([
        1 - R - Z,
        R,
        Z,
    ])

def isoparametric_mapping() -> np.array:
    """
    Isoparametric mapping natural -> physical coordinates.

    :return: functions returning the physical coordinates
    """

    sh = shape_functions()  # shape functions with substituted values
    return sp.Matrix([sp.Matrix(sh).dot(physical_nodes[:, 0]), sp.Matrix(sh).dot(physical_nodes[:, 1])])

# Calculate the derivatives by s and t, symbolically
def shape_function_derivatives() -> np.array:
    """
    Shape function derivatives.
    [0] = dNi/ds, [1] = dNi/dt
    """
    sh = shape_functions()  # N1, N2, N3, N4
    dN_ds = []
    dN_dt = []

    for i in range(len(sh)):
        dN_ds.append(sp.diff(sh[i], R))
        dN_dt.append(sp.diff(sh[i], Z))

    return dN_ds, dN_dt

print()
print('The shape functions are:')
print(shape_functions())
print()
print('The derivatives are:')
derivatives = shape_function_derivatives()
for i, dN in enumerate(derivatives[0]):
    print(f'dN{i+1}/ds = {dN}')
for i, dN in enumerate(derivatives[1]):
    print(f'dN{i+1}/dt = {dN}')

print()
print('The isoparametric mapping is:')
x_map = isoparametric_mapping()[0]
y_map = isoparametric_mapping()[1]
print(f'x = {x_map}')
print(f'y = {y_map}')

# Substitute the symbolic coordinates with numerical values
st_values = {
    R: 0.3333,  # example value for s
    Z: 0.3333,  # example value for t
}
print()
print('The isoparametric mapping with coordinate values substituted is:')
x_map_substituted = x_map.subs(st_values)
y_map_substituted = y_map.subs(st_values)
print(f'x = {x_map_substituted}')
print(f'y = {y_map_substituted}')

print()
print('The derivatives of the shape functions by s are:')
print(derivatives[0])  # derivatives by s

print()
print('The derivatives of the shape functions by t are:')
print(derivatives[1])  # derivatives by t



The shape functions are:
Matrix([[-R - Z + 1], [R], [Z]])

The derivatives are:
dN1/ds = -1
dN2/ds = 1
dN3/ds = 0
dN1/dt = -1
dN2/dt = 0
dN3/dt = 1

The isoparametric mapping is:
x = R*r2 + Z*r3 + r1*(-R - Z + 1)
y = R*z2 + Z*z3 + z1*(-R - Z + 1)

The isoparametric mapping with coordinate values substituted is:
x = 0.3334*r1 + 0.3333*r2 + 0.3333*r3
y = 0.3334*z1 + 0.3333*z2 + 0.3333*z3

The derivatives of the shape functions by s are:
[-1, 1, 0]

The derivatives of the shape functions by t are:
[-1, 0, 1]


### The shape function matrix

The shape function matrix $N$ is a matrix that contains the shape functions for each node. It has the structure:
$$
N = \begin{bmatrix}
N_i & 0 & N_j & 0 & N_m & 0 \\
0 & N_i & 0 & N_j & 0 & N_m\\
\end{bmatrix}
$$

In [6]:
def N_matrix() -> np.array:
    """
    shape function matrix N.
    It has the structure:
    [[Ni, 0, Nj, 0, Nm, 0],
     [0, Ni, 0, Nj, 0, Nm]]

    """
    _N = shape_functions()  # the shape functions

    # preparing the return matrix
    _ret = sp.Matrix(ND, ND * NNODE, lambda i, j: 0)  # ND rows, ND * NNODE columns

    # filling the matrix
    for i in range(NNODE):
        for j in range(ND):
            _ret[j, j + ND * i] = _N[i]

    return _ret

_N = N_matrix()
print("The shape function matrix N is {}:".format(_N.shape))
print()
print('The shape function matrix N for this element:')
_N_e = _N.subs(physical_coords_map)
sp.pprint(_N_e)

print('\nThe shape function matrix N for this element at node 1:')
midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0]) / 3, \
            (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1]) / 3
_N_e_node1 = _N_e.subs({'R': midpoint[0], 'Z': midpoint[1]})
sp.pprint(_N_e_node1.applyfunc(lambda x: x.round(4)))




The shape function matrix N is (2, 6):

The shape function matrix N for this element:
⎡-R - Z + 1      0       R  0  Z  0⎤
⎢                                  ⎥
⎣    0       -R - Z + 1  0  R  0  Z⎦

The shape function matrix N for this element at node 1:
⎡-2.6667     0     3.3333    0     0.3333    0   ⎤
⎢                                                ⎥
⎣   0     -2.6667    0     3.3333    0     0.3333⎦


### Jacobian matrix

In [7]:
def jacobian() -> sp.Matrix:
    """
    Calculate the jacobian matrix for the isoparametric mapping.
    """
    # The isoparametric mapping functions r(R,Z) and z(R,Z)
    mapping = isoparametric_mapping()
    r_map = mapping[0]
    z_map = mapping[1]

    # Calculate the derivatives
    dr_dR = sp.diff(r_map, R)
    dr_dZ = sp.diff(r_map, Z)
    dz_dR = sp.diff(z_map, R)
    dz_dZ = sp.diff(z_map, Z)

    return sp.Matrix([[dr_dR, dr_dZ],
                      [dz_dR, dz_dZ]])

# Calculate the jacobian matrix symbolically
J = jacobian()

# Substitute the symbolic coordinates with numerical values in the Jacobian
J_substituted = J.subs(physical_coords_map)

print('The jacobian matrix with coordinate values substituted is:')
sp.pprint(J_substituted)
print()

# Evaluate the jacobian matrix at some points
for r_eval in [0, 0.3333]:
    for z_eval in [0, 0.3333]:
        J_evaluated = J_substituted.subs({R: r_eval, Z: z_eval})
        print(f'Jacobian matrix at r={r_eval}, z={z_eval}:')
        sp.pprint(J_evaluated)
        print(f'The determinant at this point is |J|={J_evaluated.det():.2f}')
        print()

The jacobian matrix with coordinate values substituted is:
⎡1  0⎤
⎢    ⎥
⎣0  1⎦

Jacobian matrix at r=0, z=0:
⎡1  0⎤
⎢    ⎥
⎣0  1⎦
The determinant at this point is |J|=1.00

Jacobian matrix at r=0, z=0.3333:
⎡1  0⎤
⎢    ⎥
⎣0  1⎦
The determinant at this point is |J|=1.00

Jacobian matrix at r=0.3333, z=0:
⎡1  0⎤
⎢    ⎥
⎣0  1⎦
The determinant at this point is |J|=1.00

Jacobian matrix at r=0.3333, z=0.3333:
⎡1  0⎤
⎢    ⎥
⎣0  1⎦
The determinant at this point is |J|=1.00



### $\begin{bmatrix}B\end{bmatrix}$ matrix

$$\begin{bmatrix}B_i\end{bmatrix} = \begin{bmatrix}
\frac{\partial N_i}{\partial r} & 0 \\
0 & \frac{\partial N_i}{\partial z} \\
\frac{\partial N_i}{\partial z} & \frac{\partial N_i}{\partial r} \\
\frac{N_i}{r} & 0
\end{bmatrix}$$

One block of the B matrix, which relates the strain to the displacement for node i.

The order of the strain vector is:
$$\varepsilon = \begin{bmatrix}
\varepsilon_r \\
\varepsilon_\theta\\
\varepsilon_z \\
\gamma_{rz} \\
\end{bmatrix}$$

In [8]:
def calculate_B():
    """
    Calculates the complete B-matrix for the element.
    B relates the strain vector [ε_r, ε_z, γ_rz, ε_θ] to the nodal displacement vector.
    """
    J_symbolic = jacobian()
    J_det_symbolic = J_symbolic.det()

    dNdr, dNdz = shape_function_derivatives()  # dN(i)/dr, dN(i)/dz

    B = sp.zeros(4, 6)

    r_iso = isoparametric_mapping()[0]

    N_funcs = shape_functions()

    for i in range(NNODE):

        # ∂Nᵢ/∂r = (1/|J|) * [ (∂z/∂Z * ∂Nᵢ/∂R) - (∂r/∂Z * ∂Nᵢ/∂Z) ]
        dNi_dr = (1/J_det_symbolic) * (sp.diff(y_map, Z) * dNdr[i] - sp.diff(x_map, Z) * dNdz[i])

        # ∂Nᵢ/∂z = (1/|J|) * [ (∂r/∂R * ∂Nᵢ/∂Z) - (∂z/∂R * ∂Nᵢ/∂R) ]
        dNi_dz = (1/J_det_symbolic) * (sp.diff(x_map, R) * dNdz[i] - sp.diff(y_map, R) * dNdr[i])

        B[0, 2*i] = dNi_dr
        B[1, 2*i] = N_funcs[i] / r_iso
        B[2, 2*i+1] = dNi_dz
        B[3, 2*i] = dNi_dz
        B[3, 2*i+1] = dNi_dr

    return B

# Example of calculating and printing the B matrix
B_matrix = calculate_B()
# You can then substitute values as needed
B_evaluated = B_matrix.subs(physical_coords_map).subs({R: 1/3, Z: 1/3})
sp.pprint(B_evaluated.evalf().applyfunc(lambda x: round(x, 4)))

⎡-1.0   0    1.0   0    0    0 ⎤
⎢                              ⎥
⎢0.1    0    0.1   0   0.1   0 ⎥
⎢                              ⎥
⎢ 0    -1.0   0    0    0   1.0⎥
⎢                              ⎥
⎣-1.0  -1.0   0   1.0  1.0   0 ⎦


### The constitutive matrix $\begin{bmatrix}D\end{bmatrix}$

The constitutive matrix relates the stress vector to the strain vector. For plane stress conditions, it is given by:
$$
\begin{bmatrix}D\end{bmatrix} = \frac{E}{(1 - 2\nu)(1 + \nu)} \begin{bmatrix}
1 - \nu & \nu & \nu & 0 \\
\nu & 1 - \nu & \nu & 0 \\
\nu & \nu & 1 - \nu & 0 \\
0 & 0 & 0 & \frac{1 - 2\nu}{2
}
\end{bmatrix}
$$


In [9]:
def D_matrix():
    """
    The constitutive matrix for the axissymmetric tri element.
    """
    # Plane stress condition
    D = sp.zeros(4, 4)
    D[0, 0] = 1 - nu
    D[0, 1] = nu
    D[0, 2] = nu

    D[1, 0] = nu
    D[1, 1] = 1 - nu
    D[1, 2] = nu

    D[2, 0] = nu
    D[2, 1] = nu
    D[2, 2] = 1 - nu

    D[3, 3] = (1 - 2 * nu) / 2

    D *= E / ((1 - 2 * nu) * (1 + nu))

    return D

D = D_matrix()
sp.pprint(D.applyfunc(lambda x: round(x/1e9, 2)))
print(D.T == D)

⎡269.23  115.38  115.38    0  ⎤
⎢                             ⎥
⎢115.38  269.23  115.38    0  ⎥
⎢                             ⎥
⎢115.38  115.38  269.23    0  ⎥
⎢                             ⎥
⎣  0       0       0     76.92⎦
True


### Quadrature points and weights

In [10]:
gi_data = {
    1: {
        'point': ((1/3, 1/3),),  # midpoint of the triangle
        'weight': (1,)
    },
    3: {
        'point': ((1/6, 1/6), (4/6, 1/6), (1/6, 4/6), ),
        'weight': (1/3, 1/3, 1/3)
    },
}

### The stiffness matrix $\begin{bmatrix}K\end{bmatrix}$

In [26]:
NG = 1

def K_matrix() -> np.array:
    """
    Gets the stiffness matrix for the element.
    """
    K_lok = sp.zeros(NNODE * ND, NNODE * ND)  # initialize the stiffness matrix
    J = jacobian()
    J_substituted = J.subs(physical_coords_map)
    D = D_matrix()
    r_iso = isoparametric_mapping()[0]  # r(R, Z) mapping

    print('riso')
    print(r_iso)

    for p, w in zip(gi_data[NG]['point'], gi_data[NG]['weight']):
        r_gp, z_gp = p[0], p[1]  # Gauss point in natural coords

        # Evaluate B, det(J), and r at the Gauss point
        B_evaluated = B_matrix.subs(physical_coords_map).subs({R: r_gp, Z: z_gp}).evalf()
        J_evaluated = J_substituted.subs({R: r_gp, Z: z_gp}).evalf()
        det_J = J_evaluated.det()
        r_physical_at_gp = r_iso.subs(physical_coords_map).subs({R: r_gp, Z: z_gp}).evalf()

        # Calculate the stiffness matrix contribution with the 2*pi*r term
        K_contribution = 2 * sp.pi * B_evaluated.T * D * B_evaluated * r_physical_at_gp * det_J * w
        K_lok += K_contribution

    return K_lok

NG = 1
K1 = K_matrix()
K1_np = np.array(K1).astype(np.float64)

NG = 3
K3 = K_matrix()
K3_np = np.array(K3).astype(np.float64)

print(np.round(K1_np/1e12, 4))
print()
print(np.round(K3_np/1e12, 4))


riso
R*r2 + Z*r3 + r1*(-R - Z + 1)
R + 3
3.33333333333333
riso
R*r2 + Z*r3 + r1*(-R - Z + 1)
R + 3
3.16666666666667
R + 3
3.66666666666667
R + 3
3.16666666666667
[[ 6.8229  3.786  -5.5824 -1.6111 -1.7963 -2.1749]
 [ 3.786   7.2498 -2.6583 -1.6111 -1.8527 -5.6388]
 [-5.5824 -2.6583  6.1785  0.      0.298   2.6583]
 [-1.6111 -1.6111  0.      1.6111  1.6111  0.    ]
 [-1.7963 -1.8527  0.298   1.6111  1.6675  0.2417]
 [-2.1749 -5.6388  2.6583  0.      0.2417  5.6388]]

[[ 6.8549  3.786  -5.5969 -1.6111 -1.8089 -2.1749]
 [ 3.786   7.2498 -2.6583 -1.6111 -1.8527 -5.6388]
 [-5.5969 -2.6583  6.2003  0.      0.2835  2.6583]
 [-1.6111 -1.6111  0.      1.6111  1.6111  0.    ]
 [-1.8089 -1.8527  0.2835  1.6111  1.6994  0.2417]
 [-2.1749 -5.6388  2.6583  0.      0.2417  5.6388]]


In [12]:
# the stiffness matrix as a numpy array
K_np = np.array(K_e).astype(np.float64)

# the eigenvalues
eigenvalues, eigenvectors = np.linalg.eigh(K_np)
print()
print('The eigenvalues of the stiffness matrix K are:')
eigenvalues = sorted([float(x) for x in eigenvalues])

print([round(ev, 5) for ev in eigenvalues])
# print(eigenvectors.shape)
# sp.pprint(eigenvectors)

# the eigenvectors for the near-zero eigenvalues
nearzero_eigenvalues = [ev for ev in eigenvalues if abs(ev) < 1e-10]
# printing the eigenvectors for the near-zero eigenvalues
print('The eigenvectors for the near-zero eigenvalues are:')
for i, ev in enumerate(nearzero_eigenvalues):
    print(f'Eigenvalue {i + 1}: {ev:.6f}')
    print('Eigenvector:')
    print(eigenvectors[:, i])
    print()


The eigenvalues of the stiffness matrix K are:
[-0.0037, -0.00055, 125205187946.65376, 4261974693202.828, 6444292622748.29, 18337006979816.68]
The eigenvectors for the near-zero eigenvalues are:
